# 5. 쳇봇학습툴만들기

## 5.1 프로젝트구조

###### chatbot 프로젝트
 - train_tools : 챗봇 학습툴 관련파일
 - models      : 챗봇엔진에서 사용하는 딥러닝 모델관련 파일
   - intent    : 의도 분류 모델
   - ner       : 개체인식모델
 - utils       : 챗봇 개발에 필요한 유틸리티 라이브러기
 - config      : 챗봇 개발에 필요한 환경설정
 - test        : 챗봇 개발에 필용한 테스트 코드

In [2]:
!mkdir .\chatbot\train_tools\qna
!mkdir .\chatbot\models\intent
!mkdir .\chatbot\models\ner
!mkdir .\chatbot\utils
!mkdir .\chatbot\config
!mkdir .\chatbot\test

## 5.2 테이터베이스 & 테이블 생성

###### chatbot_train_data
1. id     : 학습데이터 ID
1. intent : 의도명, 의도가 없는 경우 null
1. ner    : 개체명, 개체명이 없는 경우 null
1. query  : 질문 text
1. answer : 답변 text
1. answer_image : 답변image, 없는 경우 null

In [18]:
# %%writefile .\chatbot\config\DatabaseConfig.py
DB_HOST = "127.0.0.1"
DB_USER = "root"
DB_PASSWORD = "12345"
DB_NAME = "chatbot"
DB_PORT=3306

def DatabaseConfig():
    global DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME

In [21]:
#%%writefile .\chatbot\train_tools\qna\create_train_data_table.py
import pymysql
from chatbot.config.DatabaseConfig import *

db = None

try:
    db = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )
    

    # 테이블 생성 sql 정의
    sql = '''
      CREATE TABLE IF NOT EXISTS `chatbot_train_data` (
      `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
      `intent` VARCHAR(45) NULL,
      `ner` VARCHAR(1024) NULL,
      `query` TEXT NULL,
      `answer` TEXT NOT NULL,
      `answer_image` VARCHAR(2048) NULL,
      PRIMARY KEY (`id`))
    ENGINE = InnoDB DEFAULT CHARSET=utf8
    '''
    
    # 테이블생성
    with db.cursor() as cursor:
        cursor.execute(sql)

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()

## 5.3 쳇봇학습데이터 연동하기

* 엑셀파일 train_data.xls을 chatbot_train_data에 업로드하기
* 설치 : pip install openpyxl

##### train_data.xlsx 저장
###### 컬럼* 
1. 의도(intent)    : 질문의 의도를 나타내는 텍스트, 의도가 없는 경우에는 공란
2. 개체명인식(NER) : 질문에 필요한 개체명. 개체명이 없는 경우 공란
3. 질문(Query)     : 질문테스트
4. 답변(Answer)    : 답변테스트
5. 답변이미지      : 답변에 들어갈 이미지 URL, 없는 경우 공란

In [8]:
#!pip install openpyxl
!pip show openpyxl

Name: openpyxl
Version: 3.0.10
Summary: A Python library to read/write Excel 2010 xlsx/xlsm files
Home-page: https://openpyxl.readthedocs.io
Author: See AUTHORS
Author-email: charlie.clark@clark-consulting.eu
License: MIT
Location: C:\Anaconda3\Lib\site-packages
Requires: et_xmlfile
Required-by: 


In [23]:
#%%writefile .\chatbot\train_tools\qna\load_train_data.py
import pymysql
import openpyxl

from chatbot.config.DatabaseConfig import *

# 학습데이터초기화
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
        delete from chatbot_train_data
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)    

# db에 데이터저장
def insert_data(db, xls_row):
    intent, ner, query, answer, answer_img_url = xls_row
    
    sql = '''
        INSERT chatbot_train_data(intent, ner, query, answer, answer_image) 
        values(
         '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value)
    
    # 엑셀에서 블러온 cell에 데이터가 없을 경우 null로 치환
    sql = sql.replace("None", "null")
    
    with db.cursor() as cursor:
        cursor.execute(sql)
        print(f'{query.value} 저장')
        db.commit()
        
train_file = './chatbot/train_tools/qna/train_data.xlsx'        

db = None

try:
    db = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )
    
    # 기존학습데이터초기화
    all_clear_train_data(db)
    
    # 학습파일(엑셀)로딩
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1']
    
    for row in sheet.iter_rows(min_row=2): # 헤더는 skip
        # 데이터저장
        insert_data(db, row)

    wb.close()
    
except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()    

안녕하세요 저장
반가워요 저장
{B_FOOD} 주문할게요 저장
{B_FOOD} 주문할게요 저장
{B_DT} 예약 저장
None 저장
